In [2]:
from pyqpanda import *
from pyqpanda.Visualization.circuit_draw import *

import math
import numpy as np
from fractions import Fraction

In [3]:
N = 21
a = 4

## Tool Functions

In [107]:
def draw(prog, filename=''):
    print(prog)
    
    dir_path = './circuits/5/'
    if filename != '':
        draw_qprog(prog, 'pic', filename=f'{dir_path}{filename}')

## Init Quantum Machine

In [5]:
class InitQMachine:
    def __init__(self, qubitsCount, cbitsCount, machineType = QMachineType.CPU):
        self.machine = init_quantum_machine(machineType)
        
        self.qubits = self.machine.qAlloc_many(qubitsCount)
        self.cbits = self.machine.cAlloc_many(cbitsCount)
        
        print(f'Init Quantum Machine with qubits:[{qubitsCount}] / cbits:[{cbitsCount}] Successfully')
    
    def __del__(self):
        destroy_quantum_machine(self.machine)

## Init Shor Circuit

### - init qMachine and qubits/cbits

In [6]:
phase_bits_qubits = 3
function_value_qubits = 2

total_qubits = phase_bits_qubits + function_value_qubits

In [7]:
init_machine = InitQMachine(total_qubits, phase_bits_qubits)

qubits = init_machine.qubits
cbits = init_machine.cbits

Init Quantum Machine with qubits:[5] / cbits:[3] Successfully


### - init empty qProg

In [8]:
prog = QProg()

### - First Step: apply hadamard gates

In [9]:
hadamard_circuit = create_empty_circuit()

for i in range(phase_bits_qubits):
    hadamard_circuit << H(qubits[i])
    
hadamard_circuit << BARRIER(qubits)

draw(hadamard_circuit, 'hadamard')


          ┌─┐     ! 
q_0:  |0>─┤H├ ────! 
          ├─┤     ! 
q_1:  |0>─┤H├ ────! 
          ├─┤     ! 
q_2:  |0>─┤H├ ────! 
          └─┘     ! 
q_3:  |0>──── ────! 
                  ! 
q_4:  |0>──── ────! 
                  ! 




### - Second Step: apply unitary Uf

In [90]:
unitary_circuit = create_empty_circuit()

unitary_circuit << CNOT(qubits[1], qubits[0]) << CNOT(qubits[1], qubits[2]) \
                << Toffoli(qubits[0], qubits[2], qubits[3]) \
                << CNOT(qubits[1], qubits[0]) \
                << X(qubits[0]) << Toffoli(qubits[0], qubits[3], qubits[4]) << X(qubits[0]) \
                << CNOT(qubits[2], qubits[4]) << CNOT(qubits[0], qubits[4]) \
                << CNOT(qubits[1], qubits[2])

unitary_circuit << BARRIER(qubits)

draw(unitary_circuit, 'unitary')


          ┌────┐            ┌────┐ ┌─┐     ┌─┐                   ! 
q_0:  |0>─┤CNOT├ ────── ─■─ ┤CNOT├ ┤X├ ─■─ ┤X├─── ───■────── ────! 
          └──┬─┘         │  └──┬─┘ └─┘  │  └─┘       │           ! 
q_1:  |0>────■── ───■── ─┼─ ───■── ─── ─┼─ ────── ───┼───■── ────! 
                 ┌──┴─┐  │              │            │┌──┴─┐     ! 
q_2:  |0>─────── ┤CNOT├ ─■─ ────── ─── ─┼─ ───■── ───┼┤CNOT├ ────! 
                 └────┘ ┌┴┐             │     │      │└────┘     ! 
q_3:  |0>─────── ────── ┤X├ ────── ─── ─■─ ───┼── ───┼────── ────! 
                        └─┘            ┌┴┐ ┌──┴─┐ ┌──┴─┐         ! 
q_4:  |0>─────── ────── ─── ────── ─── ┤X├ ┤CNOT├ ┤CNOT├──── ────! 
                                       └─┘ └────┘ └────┘         ! 




### - approximate toffoli gate

In [101]:
def applyToffoli(circuit, a, b, c):
    circuit << BARRIER(qubits)
    circuit << RY(qubits[c], np.pi / 4) << CNOT(qubits[b], qubits[c]) \
            << RY(qubits[c], np.pi / 4) << CNOT(qubits[a], qubits[c]) \
            << RY(qubits[c], -np.pi / 4) << CNOT(qubits[b], qubits[c]) \
            << RY(qubits[c], -np.pi / 4)
    circuit << BARRIER(qubits)
    
    return circuit

In [102]:
unitary_circuit_approximate = create_empty_circuit()

unitary_circuit_approximate << CNOT(qubits[1], qubits[0]) << CNOT(qubits[1], qubits[2])

applyToffoli(unitary_circuit_approximate, 0, 2, 3)

unitary_circuit_approximate << CNOT(qubits[1], qubits[0]) \
                << X(qubits[0])

applyToffoli(unitary_circuit_approximate, 0, 3, 4)

unitary_circuit_approximate << X(qubits[0]) \
                << CNOT(qubits[2], qubits[4]) << CNOT(qubits[0], qubits[4]) \
                << CNOT(qubits[1], qubits[2])

unitary_circuit_approximate << BARRIER(qubits)

draw(unitary_circuit_approximate, 'unitary_approximate')


          ┌────┐            !                                                                                    >
q_0:  |0>─┤CNOT├ ────── ────! ────────────── ────── ────────────── ───■── ─────────────── ────── ─────────────── >
          └──┬─┘            !                                         │                                          >
q_1:  |0>────■── ───■── ────! ────────────── ────── ────────────── ───┼── ─────────────── ────── ─────────────── >
                 ┌──┴─┐     !                                         │                                          >
q_2:  |0>─────── ┤CNOT├ ────! ────────────── ───■── ────────────── ───┼── ─────────────── ───■── ─────────────── >
                 └────┘     ! ┌────────────┐ ┌──┴─┐ ┌────────────┐ ┌──┴─┐ ┌─────────────┐ ┌──┴─┐ ┌─────────────┐ >
q_3:  |0>─────── ────── ────! ┤RY(0.785398)├ ┤CNOT├ ┤RY(0.785398)├ ┤CNOT├ ┤RY(-0.785398)├ ┤CNOT├ ┤RY(-0.785398)├ >
                            ! └────────────┘ └────┘ └────────────┘ └────┘ └────

### - Third Step: apply QFT_dagger

In [11]:
qft_dagger_circuit = create_empty_circuit()

for i in range(phase_bits_qubits - 1):
    qft_dagger_circuit << H(qubits[i])
    
    for j in range(i + 1, phase_bits_qubits):
        qft_dagger_circuit << U1(qubits[i], np.pi / (2 ** (j - i))).control(qubits[j])
    
    qft_dagger_circuit << BARRIER(qubits)

qft_dagger_circuit << H(qubits[phase_bits_qubits - 1])

draw(qft_dagger_circuit, 'qft_dagger')


          ┌─┐ ┌────────────┐ ┌────────────┐     !                        !     
q_0:  |0>─┤H├ ┤U1(1.570796)├ ┤U1(0.785398)├ ────! ─── ────────────── ────! ─── 
          └─┘ └──────┬─────┘ └──────┬─────┘     ! ┌─┐ ┌────────────┐     !     
q_1:  |0>──── ───────■────── ───────┼────── ────! ┤H├ ┤U1(1.570796)├ ────! ─── 
                                    │           ! └─┘ └──────┬─────┘     ! ┌─┐ 
q_2:  |0>──── ────────────── ───────■────── ────! ─── ───────■────── ────! ┤H├ 
                                                !                        ! └─┘ 
q_3:  |0>──── ────────────── ────────────── ────! ─── ────────────── ────! ─── 
                                                !                        !     
q_4:  |0>──── ────────────── ────────────── ────! ─── ────────────── ────! ─── 
                                                !                        !     




### - build full circuit program

In [12]:
prog << hadamard_circuit << unitary_circuit << qft_dagger_circuit

draw(prog, 'shor')


          ┌─┐     ! ┌────┐            ┌────┐ ┌─┐     ┌─┐                   ! ┌─┐ ┌────────────┐ ┌────────────┐ >
q_0:  |0>─┤H├ ────! ┤CNOT├ ────── ─■─ ┤CNOT├ ┤X├ ─■─ ┤X├─── ───■────── ────! ┤H├ ┤U1(1.570796)├ ┤U1(0.785398)├ >
          ├─┤     ! └──┬─┘         │  └──┬─┘ └─┘  │  └─┘       │           ! └─┘ └──────┬─────┘ └──────┬─────┘ >
q_1:  |0>─┤H├ ────! ───■── ───■── ─┼─ ───■── ─── ─┼─ ────── ───┼───■── ────! ─── ───────■────── ───────┼────── >
          ├─┤     !        ┌──┴─┐  │              │            │┌──┴─┐     !                           │       >
q_2:  |0>─┤H├ ────! ────── ┤CNOT├ ─■─ ────── ─── ─┼─ ───■── ───┼┤CNOT├ ────! ─── ────────────── ───────■────── >
          └─┘     !        └────┘ ┌┴┐             │     │      │└────┘     !                                   >
q_3:  |0>──── ────! ────── ────── ┤X├ ────── ─── ─■─ ───┼── ───┼────── ────! ─── ────────────── ────────────── >
                  !               └─┘            ┌┴┐ ┌──┴─┐ ┌──┴─┐         !                   

### - approximate toffoli

In [103]:
prog_approximate = create_empty_qprog()
prog_approximate << hadamard_circuit << unitary_circuit_approximate << qft_dagger_circuit

draw(prog_approximate, 'shor_approximate')


          ┌─┐     ! ┌────┐            !                                                                    >
q_0:  |0>─┤H├ ────! ┤CNOT├ ────── ────! ────────────── ────── ────────────── ───■── ─────────────── ────── >
          ├─┤     ! └──┬─┘            !                                         │                          >
q_1:  |0>─┤H├ ────! ───■── ───■── ────! ────────────── ────── ────────────── ───┼── ─────────────── ────── >
          ├─┤     !        ┌──┴─┐     !                                         │                          >
q_2:  |0>─┤H├ ────! ────── ┤CNOT├ ────! ────────────── ───■── ────────────── ───┼── ─────────────── ───■── >
          └─┘     !        └────┘     ! ┌────────────┐ ┌──┴─┐ ┌────────────┐ ┌──┴─┐ ┌─────────────┐ ┌──┴─┐ >
q_3:  |0>──── ────! ────── ────── ────! ┤RY(0.785398)├ ┤CNOT├ ┤RY(0.785398)├ ┤CNOT├ ┤RY(-0.785398)├ ┤CNOT├ >
                  !                   ! └────────────┘ └────┘ └────────────┘ └────┘ └─────────────┘ └────┘ >
q_4:  |0>──── ────

## Measure and calculate factors

In [42]:
def measure(prog):
    for i in range(phase_bits_qubits):
        prog << Measure(qubits[i], cbits[phase_bits_qubits - i - 1])

    result = run_with_configuration(prog, cbits, 1)
    print(f'  result: {result}')
    
    return result

In [43]:
def QPE(result):
    phase = int(list(result)[0], 2) / (2 ** phase_bits_qubits)
    print(f'   - corresponding phase: {phase}')
    
    return phase

In [44]:
def calculatePeriod(phase):
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    print(f'\n  Rewrite to fraction: {frac}, thus r = {r}')
    
    return r

In [52]:
def calculateFactors(r):
    guesses = [math.gcd(a ** (r // 2) - 1, N), math.gcd(a ** (r // 2) + 1, N)]
    print(f'  calculate final guesses: {guesses}')
    
    factors = []
    for num in guesses:
        if num not in [1, N]:
            factors.append(num)
            print(f'[Find non-trivial factor: {num}]')
    
    if len(factors) == 0:
        print('[Failed]')
        return False, []
    elif len(factors) == 1:
        factors.append(N // factors[0])
    
    return True, sorted(factors)

### - final execute

In [77]:
def execute(prog):
    attempt = 1
    
    while True:
        print(f'Execute shor algorithm - attempt time: [{attempt}]')

        result = measure(prog)
        phase = QPE(result)

        # period
        r = calculatePeriod(phase)

        ok, factors = calculateFactors(r)
        if ok:
            print(f'\nFactors of {N} are: {factors} (total run times: {attempt})')
            break
            
        attempt += 1
        print('\n' + '-' * 36 + '\n')

In [104]:
execute(prog)

Execute shor algorithm - attempt time: [1]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, thus r = 1
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 5/8, thus r = 8
  calculate final guesses: [3, 1]
[Find non-trivial factor: 3]

Factors of 21 are: [3, 7] (total run times: 2)


In [106]:
execute(prog_approximate)

Execute shor algorithm - attempt time: [1]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, thus r = 1
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'011': 1}
   - corresponding phase: 0.375

  Rewrite to fraction: 3/8, thus r = 8
  calculate final guesses: [3, 1]
[Find non-trivial factor: 3]

Factors of 21 are: [3, 7] (total run times: 2)
